In [1]:
import numpy as np
import cv2

In [2]:
nsamples = 1000
nvar = 5;
nfact = 2;
## F3 = F1 + F2
## F4 = F3
## F5 = F4 - F2

# Creating a 5X1000 data matrix
F = np.zeros(shape=(nvar,nsamples));
F[0,:] = np.random.random(nsamples)*500+500;
F[1,:] = np.random.random(nsamples)*300+300;
F[2,:] = F[0,:] + F[1,:]; + np.random.random(nsamples)*40-20;
F[3,:] = F[2,:] + np.random.random(nsamples)*40-20;
F[4,:] = F[3,:] - F[1,:] + np.random.random(nsamples)*40-20;

# Transposing F
#F = np.transpose(F);

In [3]:
U, S, V = np.linalg.svd(F,full_matrices=False)
#np.matmul(np.matmul(U,np.diag(S)),V) - F

In [4]:
#U.shape,S.shape,V.shape

In [5]:
Amat = np.zeros(shape=(nvar-nfact,nvar));
#Amat[0,:] = U[:,2].T
#Amat[1,:] = U[:,3].T
#Amat[2,:] = U[:,4].T

Amat[:nvar-nfact,:] = U[:,nfact:nvar].T

Aindep = Amat[:,0:2]
Adep = Amat[:,2:5]

In [6]:
reg = -np.matmul((np.linalg.inv(np.matmul(Adep.T,Adep))),(np.matmul(Adep.T,Aindep)))
#reg

In [8]:
W2 = V.T[:, :2]
F2D = F.dot(W2)

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X2D = pca.fit_transform(F)

In [10]:
#pca.components_.T[:,0] 

In [11]:
#print(pca.explained_variance_ratio_)

In [12]:
pca = PCA()
pca.fit(F)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.99) + 1
#print(d,cumsum)

In [13]:
pca = PCA(n_components=0.95)
F_reduced = pca.fit_transform(F)

In [14]:
pca = PCA(n_components = 2)
F_reduced = pca.fit_transform(F)
F_recovered = pca.inverse_transform(F_reduced)

In [82]:
img = cv2.imread("./a.png",0);

In [83]:
pca = PCA()
pca.fit(img)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
d

41

In [84]:
pca = PCA(n_components = d)
F_reduced = pca.fit_transform(img)
F_recovered = pca.inverse_transform(F_reduced)

In [85]:
cv2.imwrite('b.png',F_recovered)

True

In [103]:
from sklearn.decomposition import IncrementalPCA
n_batches = 12
inc_pca = IncrementalPCA(n_components=d)
for X_batch in np.array_split(img, n_batches):
    inc_pca.partial_fit(X_batch)
    
F_reduced = inc_pca.transform(img)

In [104]:
F_recovered = inc_pca.inverse_transform(F_reduced)
cv2.imwrite('c.png',F_recovered)

True